# Chapter 2. 문장을 작은 단위로 쪼개기  

자연어 처리의 첫 단추는 자연어 문장을 적은 단위인 토큰으로 분석하는 과정입니다. 이 장에서는 바이트 페어 인코딩을 중심으로 이론을 살펴봅시다. 이어 어휘 집합을 구축하고 이 어휘 집합으로 잘게 쪼갠 토큰들을 정수로 변환하는 실습을 진행합니다.

## <2-1 토큰화란?>  

***토큰화(tokenization)*** 란 문장을 토큰 시퀀스로 나누는 과정입니다. 수행 대상에 따라 ***문자, 단어, 서브워드*** 등 세 가지 방법이 있습니다. 이번 절에서는 각 토큰화 방법의 장단점을 살펴보겠습니다.


### 토큰화란 무엇일까?
---
이 책에서 다루는 데이터의 기본 단위는 텍스트 형태의 문장입니다. 트랜스포머 모델은 토큰 시퀀스를 입력받으므로 문장에 토큰화를 수행해 줘야 합니다.  

토큰화를 수행하는 프로그램을 ***토크나이저(tokenizer)*** 라고 합니다. 대표적인 한국어 토크나이저로는 은전한닢(mecab), 꼬꼬마(kkma) 등이 있습니다. 이들은 언어 전문가들이 토큰화해 놓은 데이터를 학습해 최대한 전문적인 분석 결과와 비슷하게 토큰화를 수행합니다.  

이들 분석기는 토큰화뿐만 아니라 ***품사 부착(Part-of-Speech tagging)*** 까지 수행하므로 토큰화 개념을 넢은 의미로 해석할 떄는 토큰 나누기에 품사 부착까지 일컫는 경우도 종종 있습니다.

### 단어 단위 토큰화
---
토큰화 방식에는 여러 가지가 있습니다. 우선 ***단어*** (어절) 단위로 토큰화를 수행할 수 있습니다. 가장 쉽게는 공백으로 분리할 수 있습니다.  
예를 들면 다음과 같습니다.  





><center>어제 카페 갔었어 ➔ 어제, 카페, 갔었어</center>  
>

>
><center>어제 카페 갔었는데요 ➔ 어제, 카페, 갔었는데요</center>

공백으로 분리하면 별도로 토크나이저를 쓰지 않아도 된다는 장점이 있지만, ***어휘 집합(vocabulary)*** 의 크기가 매우 커질 수 있습니다. ***갔었어, 갔었는데요*** 처럼 표현이 살짝만 바뀌어도 모든 경우의 수가 어휘 집합에 포함돼야 하기 때문입니다.  

만약 학습된 토크나이저를 사용하면 어휘 집합의 크기가 커지는 것을 조금 완화할 수는 있습니다. 예를 들어 같은 문장을 은전한닢으로 토큰화하면 다음과 같습니다. 예시가 적어서 효과가 도드라져 보이지는 않지만, 의미 있는 단위(***갔었*** )로 토큰화해 어휘 집합이 급격하게 커지는 것을 다소 막을 수 있습니다.

><center>어제 카페 갔었어 ➔ 어제, 카페, 갔었, 어</center>  
>

>
><center>어제 카페 갔었는데요 ➔ 어제, 카페, 갔었, 는데요</center>

그렇지만 은전한닢 같은 토크나이저를 사용하더라도 어휘 집합 크기가 지나치게 커지는 것은 막기 어렵습니다. 보통 언어 하나로 모델을 구축할 때 어휘 집합 크기는 10만 개를 훌쩍 넘는 경우가 다반사입니다. 어휘 집합 크기가 커지면 그만큼 모델 학습이 어려워질 수 있습니다.

### 문자 단위 토큰화
---
토큰화를 단어 대신 ***문자*** 단위로 고려해 볼 수 있습니다. 한글 위주로 표현된 데이터로 언어 모델을 만든다고 하면, 한글로 표현할 수 있는 글자는 모두 1만 1,172개이므로 알파벳, 숫자, 기호 등을 고려해도 어휘 집합 크기는 기껏해야 1만 5,000개를 넘기 어렵습니다. 게다가 해당 언어의 모든 문자를 어휘 집합에 포함하므로 미등록 토큰 문제로부터 자유롭습니다.  

cf) 미등록 토큰(unknown token)이란 어휘 집합에 없는 토큰을 가리키는데요. 주로 신조어 등에서 발생합니다.  

하지만 문자 단위로 토큰화를 수행할 경우 분명히 단점도 있습니다. 우선 각 문자 토큰은 의미있는 단위가 되기 어렵습니다. 예를 들어 ***어제*** 의 ***어*** 와 어미인 ***어*** 의 구분이 사라진다든지 말입니다.

><center>어제 카페 갔었어 ➔ 어, 제, 카, 페, 갔, 었, 어</center>  
>

>
><center>어제 카페 갔었는데요 ➔ 어, 제, 카, 페, 갔, 었, 는, 데, 요</center>

이뿐만 아니라 문자 단위 토큰화는 앞의 단어 단위와 비교할 때 분석 결과인 토큰 시퀀스의 길이가 상대적으로 길어졌음을 확인할 수 있습니다. 언어 모델에 입력할 토큰 시퀀스가 길면 모델이 해당 문장을 학습하기가 어려워지고 결과적으로 성능이 떨어지게 됩니다.

### 서브워드 단위 토큰화
---
***서브워드(subword)*** 단위 토큰화는 단어와 문자 단위 토큰화의 중간에 있는 형태입니다. 둘의 장점만을 취한 형태죠.  
즉, 어휘 집합 크기가 지나치게 커지지 않으면서도 미등록 토큰 문제를 피하고, 분석된 토큰 시퀀스가 너무 길어지지 않게 합니다. 대표적인 서브워드 단위 토큰화 기법이라면 바이트 페어 인코딩을 들 수 있는데 다음 절에서 자세하게 살펴보겠습니다.